<a href="https://colab.research.google.com/github/Viktory4121/MAPI/blob/main/solving_SLAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Алгоритм Фаддеева

In [ ]:
import numpy as np

#A = np.array([[-9,3,4,1],[3,0,1,-10]])
#A = np.array([[8,-5,1,4,-9],[2,5,-1,4,2],[-1,-2,-6,9,5],[6,2,0,-7,2],[6,2,5,9,1],[-5,-1,4,8,-2]])
#A = np.array([[10,-62,11,6],[0,24,-5,-7],[1,8,4,12]])
#A = np.array([[-1,2,5],[5,2,7],[0,-7,-1],[6,5,4],[1,6,2],[1,2,3],[-7,-4,-1]])
A = np.array([[1,2,7,-2,1],[6,0,4,-2,5],[-4,-8,3,2,1],[7,0,-4,3,2]])

def print_matrix(A):
  for row in A:
    for i in row:
      #print("{} & ".format(str(round(i))), end = "" )
      print("{}\t\t".format(str(round(i))), end = "" )
    print()

def print_float_matrix(A):
  for row in A:
    for i in row:
      #print("{:6f} & ".format(i), end = "" )
      print("{:6f}\t\t".format(i), end = "" )
    print()

#--------------------------------
n = len(A[0])
A_T = A.transpose()
A_T_A = A_T.dot(A) #перемножены транспонированная матрица и обычная
F = np.diag(np.diag(np.ones([n,n]))) #единичная матрица
mu = sum(np.diag(A_T_A)) #это фи1 (сумма значений матрицы A_T_A по диагонали)
print("A =")
print_matrix(A)
print("Шаг 1")
print("Phi = I;")
print("phi_1 = ", mu)
#print("det(A) = ", np.linalg.det(A))
#--------------------------------

def pseudo(A_T_A, F, mu, A_T):
    n = len(F)
    I = np.diag(np.diag(np.ones([n,n])))
    i = 2
    while mu != 0:
        if sum(np.diag(A_T_A.dot(mu*I - A_T_A.dot(F))))/float(i) == 0:
            _mu = mu
            _F = F
        F = mu*I - A_T_A.dot(F)
        mu = sum(np.diag(A_T_A.dot(F)))/float(i)
        
        print("\nШаг", i)
        print("Phi:")
        print_matrix(F)
        #print("\nA_T_A*F:\n", A_T_A.dot(F))
        print("phi\n", mu)
        
        i+=1

    print("\n\nРЕЗУЛЬТАТ:\nA*: 1/", _mu)
    A_psevv = _F.dot(A_T)
    print_matrix(A_psevv)

    print("\nили A*:")
    print_float_matrix(A_psevv/_mu)
    return(A_psevv/_mu, _mu)

A_psev, muu = pseudo(A_T_A, F, mu, A_T)

A =
1		2		7		-2		1		
6		0		4		-2		5		
-4		-8		3		2		1		
7		0		-4		3		2		
Шаг 1
Phi = I;
phi_1 =  312

Шаг 2
Phi:
210		-34		9		1		-41		
-34		244		10		20		6		
9		10		222		28		-22		
1		20		28		291		4		
-41		6		-22		4		281		
phi
 31668.0

Шаг 3
Phi:
13167		-4964		1670		-403		-7737		
-4964		16768		1662		4730		2532		
1670		1662		13137		5749		-4399		
-403		4730		5749		26758		503		
-7737		2532		-4399		503		25174		
phi
 1078104.0

Шаг 4
Phi:
235690		-147908		77493		-61838		-368136		
-147908		233068		50160		287730		207996		
77493		50160		185174		264421		-188147		
-61838		287730		264421		773367		9864		
-368136		207996		-188147		9864		728909		
phi
 3925537.0

Шаг 5
Phi:
643204		-626362		35288		-726612		-1090720		
-626362		609961		-34364		707586		1062160		
35288		-34364		1936		-39864		-59840		
-726612		707586		-39864		820836		1232160		
-1090720		1062160		-59840		1232160		1849600		
phi
 0.0


РЕЗУЛЬТАТ:
A*: 1/ 3925537.0
237865		7108		-18829		418072		
301884		-222288		-338976		43186		
757042		-263923

#Проверки Мура-Пенроуза

In [ ]:
#Проверка 1 ((A A^+)^T = A A^+)
print("Проверка 1.")
Check1_1 = (A.dot(A_psev)).transpose()
Check1_2 = A.dot(A_psev)
print("(A A^+)^T:")
print_float_matrix(Check1_1)
print()
print("A A^+:")
print_float_matrix(Check1_2)

#Проверка 2 ((A^+ A)^T = A^+ A)
#A_psev1 = A_psev.dot(muu)
print("\n\nПроверка 2.")
Check2_1 = (A_psev.dot(A)).transpose()
Check2_2 = A_psev.dot(A)
print("(A^+ A)^T:")
print_matrix(Check2_1)
print()
print("A^+ A:")
print_matrix(Check2_2)

#Проверка 3 (A A^+ A = A)
print("\n\nПроверка 3.")
Check3_1 = A.dot(A_psev).dot(A)
Check3_2 = A
print("A A^+ A:")
print_matrix(Check3_1)
print("A:")
print_matrix(Check3_2)

#Проверка 4 (A^+ A A^+ = A^+)
print("\n\nПроверка 4.")
Check4_1 = A_psev.dot(A).dot(A_psev)
Check4_2 = A_psev
print("A^+ A A^+:")
print_float_matrix(Check4_1)
print()
print("A^+ A A^+:")
print_float_matrix(Check4_2)

Проверка 1.
(A A^+)^T:
1.000000		0.000000		0.000000		0.000000		
0.000000		1.000000		-0.000000		-0.000000		
0.000000		0.000000		1.000000		0.000000		
0.000000		0.000000		-0.000000		1.000000		

A A^+:
1.000000		0.000000		0.000000		0.000000		
0.000000		1.000000		0.000000		0.000000		
0.000000		-0.000000		1.000000		-0.000000		
0.000000		-0.000000		0.000000		1.000000		


Проверка 2.
(A^+ A)^T:
1		0		0		0		0		
0		1		0		0		0		
0		0		1		0		0		
0		0		0		1		0		
0		0		0		0		1		

A^+ A:
1		0		0		0		0		
0		1		0		0		0		
0		0		1		0		0		
0		0		0		1		0		
0		0		0		0		1		


Проверка 3.
A A^+ A:
1		2		7		-2		1		
6		0		4		-2		5		
-4		-8		3		2		1		
7		0		-4		3		2		
A:
1		2		7		-2		1		
6		0		4		-2		5		
-4		-8		3		2		1		
7		0		-4		3		2		


Проверка 4.
A^+ A A^+:
0.060594		0.001811		-0.004797		0.106501		
0.076903		-0.056626		-0.086351		0.011001		
0.192851		-0.067232		0.047118		0.055718		
0.210850		-0.206534		0.075244		0.216347		
-0.142654		0.168999		-0.001841		-0.085836		

A^+ A A^+:
0.060594		0.001811		-0.00479